In [42]:
import pandas as pd
import os
import replicaEVSE.datautils as datautils

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

mode = 'PRIVATE_AUTO'
test = False
datadir = '../../data/'

In [28]:
#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

# Dwell time

### test how to calculate stop duration or dwell time for all vehicles in the priave auto/ldv subset of the replica trip data. 

We want to do this in a more efficient manner than looping over each person in the dataset. This is still slow, 3 hours on 512Gb ram VM. 

The code to run this on the full dataset is in /scrips/add_dwell_time.py

In [50]:
groupby_df_stop_dur = df.head(100000).groupby(['person_id', 'weekday']).apply(datautils.calculate_stop_duration).reset_index(drop=True)

In [38]:
groupby_df_stop_dur[['person_id', 'start_time', 'end_time', 'stop_duration', 'weekday']].reset_index(drop=True).head(23)

,person_id,start_time,end_time,stop_duration,weekday
0,10000002909730821040,0 days 13:28:23,0 days 13:51:27,0 days 07:12:54,saturday
1,10000002909730821040,0 days 21:04:21,0 days 21:38:18,0 days 15:50:05,saturday
2,10000002909730821040,0 days 08:20:42,0 days 08:39:00,0 days 00:33:45,thursday
3,10000002909730821040,0 days 09:12:45,0 days 09:19:56,0 days 04:09:34,thursday
4,10000002909730821040,0 days 13:29:30,0 days 13:39:45,0 days 00:58:40,thursday
5,10000002909730821040,0 days 14:38:25,0 days 14:48:33,0 days 01:07:11,thursday
6,10000002909730821040,0 days 15:55:44,0 days 16:10:30,0 days 05:57:23,thursday
7,10000002909730821040,0 days 22:07:53,0 days 22:16:46,0 days 00:22:28,thursday
8,10000002909730821040,0 days 22:39:14,0 days 23:06:23,0 days 09:14:19,thursday
9,10000003183049770926,0 days 11:35:32,0 days 11:44:09,0 days 01:35:59,saturday


## test the parallel implementation

    - single computation is 1min 35s
    - parallel compution is 40s



In [51]:
import joblib
import replicaEVSE.datautils as simdu
groups = df.head(100000).groupby(['person_id', 'weekday'])
out = joblib.Parallel(verbose=10, n_jobs=60)(joblib.delayed(simdu.calculate_stop_duration)(group) for name, group in groups)
df_paralell = pd.concat(out).reset_index(drop=True)

[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.


[Parallel(n_jobs=60)]: Done   8 tasks      | elapsed:   10.3s
[Parallel(n_jobs=60)]: Done  25 tasks      | elapsed:   10.4s
[Parallel(n_jobs=60)]: Done  42 tasks      | elapsed:   10.4s
[Parallel(n_jobs=60)]: Done  61 tasks      | elapsed:   10.4s
[Parallel(n_jobs=60)]: Batch computation too fast (0.1951s.) Setting batch_size=2.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:   10.5s
[Parallel(n_jobs=60)]: Done 101 tasks      | elapsed:   10.5s
[Parallel(n_jobs=60)]: Done 122 tasks      | elapsed:   10.5s
[Parallel(n_jobs=60)]: Done 145 tasks      | elapsed:   10.6s
[Parallel(n_jobs=60)]: Done 168 tasks      | elapsed:   10.6s
[Parallel(n_jobs=60)]: Done 193 tasks      | elapsed:   10.6s
[Parallel(n_jobs=60)]: Done 218 tasks      | elapsed:   10.7s
[Parallel(n_jobs=60)]: Batch computation too fast (0.1751s.) Setting batch_size=4.
[Parallel(n_jobs=60)]: Done 251 tasks      | elapsed:   10.7s
[Parallel(n_jobs=60)]: Done 305 tasks      | elapsed:   10.8s
[Parallel(n_jobs=60)]: Done 

In [52]:
df_paralell[['person_id', 'start_time', 'end_time', 'stop_duration', 'weekday']].head(23)

,person_id,start_time,end_time,stop_duration,weekday
0,10000002909730821040,0 days 13:28:23,0 days 13:51:27,0 days 07:12:54,saturday
1,10000002909730821040,0 days 21:04:21,0 days 21:38:18,0 days 15:50:05,saturday
2,10000002909730821040,0 days 08:20:42,0 days 08:39:00,0 days 00:33:45,thursday
3,10000002909730821040,0 days 09:12:45,0 days 09:19:56,0 days 04:09:34,thursday
4,10000002909730821040,0 days 13:29:30,0 days 13:39:45,0 days 00:58:40,thursday
5,10000002909730821040,0 days 14:38:25,0 days 14:48:33,0 days 01:07:11,thursday
6,10000002909730821040,0 days 15:55:44,0 days 16:10:30,0 days 05:57:23,thursday
7,10000002909730821040,0 days 22:07:53,0 days 22:16:46,0 days 00:22:28,thursday
8,10000002909730821040,0 days 22:39:14,0 days 23:06:23,0 days 09:14:19,thursday
9,10000003183049770926,0 days 11:35:32,0 days 11:44:09,0 days 01:35:59,saturday


## verify that the script worked

In [46]:
new_df = pd.read_parquet(datadir+'wa_ldv_trips_with_county_and_dwell_time.parquet')

In [47]:
new_df.shape, df.shape

(27124460, 70)

In [48]:
new_df.head(23)

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,destination_county,stop_duration
0,15608817749489036848,10000002909730821040,PRIVATE_AUTO,WORK,HOME,0 days 13:28:23,0 days 13:51:27,9.548814,nan,530250111002,47.098457,-119.319023,530250110001,47.131321,-119.203190,residential,single_family,residential,single_family,commercial,retail,commercial,retail,47.085370,-119.319090,47.132725,-119.213000,saturday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,WORK,"Grant County, WA",0 days 07:12:54
1,10652465939061186874,10000002909730821040,PRIVATE_AUTO,HOME,WORK,0 days 21:04:21,0 days 21:38:18,9.284612,nan,530250110001,47.131321,-119.203190,530250111002,47.098457,-119.319023,commercial,retail,commercial,retail,residential,single_family,residential,single_family,47.132725,-119.213000,47.085370,-119.319090,saturday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,HOME,"Grant County, WA",0 days 15:50:05
2,18336634547462657450,10000002909730821040,PRIVATE_AUTO,SOCIAL,HOME,0 days 08:20:42,0 days 08:39:00,2.897196,nan,530250111002,47.098457,-119.319023,530250111003,47.117839,-119.303234,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.085370,-119.319090,47.118737,-119.298334,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",0 days 00:33:45
3,1694659094182985471,10000002909730821040,PRIVATE_AUTO,HOME,SOCIAL,0 days 09:12:45,0 days 09:19:56,2.867602,nan,530250111003,47.117839,-119.303234,530250111002,47.098457,-119.319023,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.118737,-119.298334,47.085370,-119.319090,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,HOME,"Grant County, WA",0 days 04:09:34
4,1649672869607162870,100000029097308210